In [11]:
import numpy as np
import pandas as pd

In [2]:
customers=pd.read_csv('Downloads/ze/Customers.csv')

In [3]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    200 non-null    object
 1   CustomerName  200 non-null    object
 2   Region        200 non-null    object
 3   SignupDate    200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [4]:
products=pd.read_csv('Downloads/ze/Products.csv')

In [5]:
Transactions=pd.read_csv('Downloads/ze/Transactions.csv')

In [6]:
merged_data = Transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',  
    'Quantity': 'sum',    
    'Region': 'first',    
}).reset_index()
customer_features = pd.get_dummies(customer_features, columns=['Region'], drop_first=True)
features = customer_features.drop(columns=['CustomerID']).values
similarity_matrix = cosine_similarity(features)
lookalike_results = {}

In [12]:
lookalike_results = {}
for i in range(20):  
    similar_indices = np.argsort(-similarity_matrix[i])[1:4]
    similar_customers = customer_features.iloc[similar_indices]['CustomerID']
    scores = similarity_matrix[i][similar_indices]
    lookalike_results[customer_features.iloc[i]['CustomerID']] = list(zip(similar_customers, scores))

In [17]:
lookalike_df = pd.DataFrame([
    {'CustomerID': cust_id, 
     'Lookalikes': str(lookalikes)} 
    for cust_id, lookalikes in lookalike_results.items()
])
lookalike_df.to_csv('Lookalike.csv', index=False)

In [18]:
lookalike_df = pd.DataFrame.from_dict(lookalike_results, orient='index')
lookalike_df.to_csv('Lookalike.csv', header=False)

In [19]:
print("Lookalike Model Output:")
lookalike_df.head(10)

Lookalike Model Output:


,0,1,2
C0001,"(C0153, 0.9999999961694737)","(C0011, 0.9999999953170958)","(C0191, 0.9999999951500694)"
C0002,"(C0043, 0.9999999978943409)","(C0034, 0.9999999337353555)","(C0003, 0.9999999058828556)"
C0003,"(C0031, 0.9999999800265182)","(C0025, 0.9999999601601218)","(C0071, 0.9999999300373303)"
C0004,"(C0039, 0.9999999975700483)","(C0195, 0.9999999946665435)","(C0169, 0.9999999926262992)"
C0005,"(C0173, 0.9999999999970868)","(C0145, 0.9999999997034683)","(C0143, 0.9999999994532175)"
C0006,"(C0126, 0.9999999998071625)","(C0171, 0.9999999961651772)","(C0118, 0.9999999958529029)"
C0007,"(C0146, 0.9999999999408112)","(C0021, 0.9999999993668264)","(C0138, 0.9999999974920598)"
C0008,"(C0160, 0.9999999891068195)","(C0047, 0.9999999817205354)","(C0038, 0.9999999810851375)"
C0009,"(C0198, 0.9999999910572125)","(C0063, 0.9999998843464756)","(C0167, 0.9999997596785154)"
C0010,"(C0097, 0.9999997902154776)","(C0049, 0.9999996741872684)","(C0091, 0.9999995921568353)"
